In [4]:
# movie 데이터 로드
import pandas as pd
df = pd.read_csv("./data/kdrama.csv")

filter_df = df.drop(["Aired Date","Aired On","Duration","Content Rating","Production companies", "Rank"], axis=1);

import chromadb
client = chromadb.PersistentClient(path="./data")

collection = client.get_or_create_collection(
    name="k-drama",
    metadata={"hnsw:space": "cosine"}
)

# 데이터 준비
# 인덱스
ids = []
# 메타데이터
doc_meta = []
# 벡터로 변환 저장할 텍스트 데이터로 ChromaDB에 Embedding 데이터가 없으면 자동으로 벡터로 변환해서 저장한다고 한다. 
documents = []

for idx in range(len(filter_df)):
    item = filter_df.iloc[idx]
    id = item['Name'].lower().replace(' ','-')
    document = f"{item['Name']}: {item['Synopsis']} : {str(item['Cast']).strip().lower()} : {str(item['Genre']).strip().lower()}"
    meta = {
        "rating" : item['Rating']
    }
    
    ids.append(id)
    doc_meta.append(meta)
    documents.append(document)

# DB 저장
collection.add(
    documents=documents,
    metadatas=doc_meta,
    ids=ids
)
# DB 쿼리
collection.query(
    query_texts=["medical drama about doctors"],
    n_results=5,
)

{'ids': [['doctor-john',
   'dr.-romantic-2',
   'hospital-playlist',
   'doctor-prisoner',
   "god's-quiz:-reboot"]],
 'distances': [[0.3436622619628906,
   0.45732051134109497,
   0.4621430039405823,
   0.4815513491630554,
   0.5326414392990194]],
 'metadatas': [[{'rating': 8.5},
   {'rating': 8.7},
   {'rating': 9.1},
   {'rating': 8.4},
   {'rating': 8.3}]],
 'embeddings': None,
 'documents': [['Doctor John: “Doctor John” is a medical drama about doctors specializing in pain management. In a refreshing take on the genre, “Doctor John” will portray the doctors’ search for the cause of their patients’ mysterious pain as a thrilling chase, almost like a detective hunting down the perpetrator behind an unsolved crime.  : ji sung, lee se young, lee kyu hyung, hwang hee, jung min ah, kwon hwa woon : mystery,  romance,  life,  medical',
   'Dr. Romantic 2: A “real doctor” story set in a small, humble hospital called Doldam Hospital. It is a story about people who meet Kim Sa Bu, a genius 

In [6]:
vector_res = collection.query(
    query_texts=["medical drama about doctors"],
    n_results=5,
)

srchres = []
for v in vector_res['documents'][0]:
    item = v.split(':')
    srchres.append({
        "title" : item[0].strip(),
        "synopsis" : item[1].strip(),
        "cast" : item[2].strip(),
        "genre" : item[3].strip(),
    })

In [7]:
srchres

[{'title': 'Doctor John',
  'synopsis': '“Doctor John” is a medical drama about doctors specializing in pain management. In a refreshing take on the genre, “Doctor John” will portray the doctors’ search for the cause of their patients’ mysterious pain as a thrilling chase, almost like a detective hunting down the perpetrator behind an unsolved crime.',
  'cast': 'ji sung, lee se young, lee kyu hyung, hwang hee, jung min ah, kwon hwa woon',
  'genre': 'mystery,  romance,  life,  medical'},
 {'title': 'Dr. Romantic 2',
  'synopsis': 'A “real doctor” story set in a small, humble hospital called Doldam Hospital. It is a story about people who meet Kim Sa Bu, a genius doctor, and discover “real romance.”',
  'cast': 'han seok kyu, ahn hyo seop, lee sung kyung, kim joo heon, shin dong wook, so ju yeon',
  'genre': 'romance,  drama,  medical,  melodrama'},
 {'title': 'Hospital Playlist',
  'synopsis': "The stories of people going through their days that are seemingly ordinary but actually spe